In [3]:
class link:
    def __init__(self, a0, a1):
        self.travel_time = 0 # the travel time of the link
        self._flow = 0 # the flow on the link
        self._paths = [] # the paths using this link
        self._a0 = a0
        self._a1 = a1
        
    def add_path(path):
        self._paths.append(path)
    
    def update_flow():
        self._flow = 0
        for path in self._paths:
            self._flow += path.flow
    
    def update_travel_time():
        self.update_flow()
        self.travel_time = self._a0 + self._a1 * self._flow

In [4]:
class path:
    def __init__(self, links):
        self._travel_time = 0 # the travel time of the paths
        self.flow = 0 # the flow on the paths
        self._links = links # the paths using this link
        for link in links:
            link.add_path(self)
    
    def update_travel_time():
        self._travel_time = 0
        for link in self._links:
            self._travel_time += link.travel_time